In [ ]:
import labelbox as lb
from API_key_personal import PERSONAL_API_KEY
import os

client = lb.Client(PERSONAL_API_KEY)

In [ ]:
PROJECT_ID = "clqsva8cw1qk407wn7vpy9xn5"

In [ ]:
#Export data rows that are done
project = client.get_project(PROJECT_ID)

export  = project.export_v2(filters={"workflow_status":"Done"})

export_json = export.result

print(len(export_json))

In [ ]:
#Get ontology

ontology = project.ontology()

print(ontology.uid)

In [ ]:
#Create model

model = client.create_model(name="Animal_ML", ontology_id=project.ontology().uid)

In [ ]:
#Create model run

client.enable_experimental = True

model_run = model.create_model_run(name="run_1")

In [ ]:
model_run._upsert_labels_by_project_id(project_id=project.uid, timeout_seconds=6000)